In [ ]:
import pandas as pd
import glob
import os


In [ ]:
#create a list of filenames we need to combine
folder_path = 'ülkeler müzik'
file_pattern = os.path.join(folder_path, '*.csv')
all_files = glob.glob(file_pattern)

print(f"Found {len(all_files)} CSV files to process.")


In [ ]:
all_dataframes = []

for file_path in all_files:
    # 1. Read the CSV file
    try:
        df = pd.read_csv(file_path,header=1)
    except Exception as e:
        print(f"Skipping file {file_path} due to error: {e}")
        continue

    # 2. Extract the country name from the filename (assuming filenames are like 'US_top200.csv')
    file_name = os.path.basename(file_path)
    country_code = file_name.split(' ')[3] # Adjust this based on your naming convention
    if country_code == "New" or country_code =="South" or country_code =="Ivory" or country_code=="United":
        country_code= file_name.split(' ')[3]+" "+file_name.split(' ')[4]

    # 3. Add a new column to the DataFrame indicating the source country
    df['Country'] = country_code
    
    # 4. Add the processed DataFrame to the list
    all_dataframes.append(df[:30])

# 5. Concatenate all DataFrames into one single DataFrame
master_df = pd.concat(all_dataframes, ignore_index=True)

print(f"\nMaster DataFrame created with {len(master_df)} total rows.")

In [ ]:
master_df.to_excel("my_data.xlsx", index=False)

In [ ]:
#to check manually
df = master_df

# 2. Select only the Artist and Title columns
unique_songs = df[['Artist', 'Title']]

# 3. Drop duplicates to keep only unique combinations
unique_songs = unique_songs.drop_duplicates()

# 4. Save to a new Excel file
output_file = 'unique_songs_list.xlsx'
unique_songs.to_excel(output_file, index=False)

print(f"Success! Found {len(unique_songs)} unique songs.")
print(f"Saved list to: {output_file}")

In [ ]:
#create a unique songs list i can annotate
df = master_df

# 2. Select only the Artist and Title columns
unique_songs = df[['Artist', 'Title']]

# 3. Drop duplicates to keep only unique combinations
unique_songs = unique_songs.drop_duplicates()

# 4. Save to a new Excel file
output_file = 'unique_songs_list.xlsx'
unique_songs.to_excel(output_file, index=False)

print(f"Success! Found {len(unique_songs)} unique songs.")
print(f"Saved list to: {output_file}")

I tried to use Spotify, Shazam and bla bla API which all didnt work / returned NA or wrong values for almost all songs. Additionally i didnt have enough request keys to finish my project with any publicly available API. So i had to rely on AI anotation to add genre and mood values which I checked with random sampling. This process created the annotated_unique_songs_list.xlsx

In [ ]:
annotated = pd.read_excel("annotated_unique_songs_list.xlsx")
annotated

In [ ]:


# 1. Load your Main Data
# Based on your upload, it seems to be a CSV file
my_data = pd.read_excel("my_data.xlsx")

# 2. Load the Annotated Data (The one we built with Genre/Mood)
annotated_df = pd.read_excel("annotated_unique_songs_list.xlsx")

# 3. Clean the text columns to ensure they match
# (Removes accidental spaces like "Taylor Swift " vs "Taylor Swift")
my_data['Artist'] = my_data['Artist'].astype(str).str.strip()
my_data['Title'] = my_data['Title'].astype(str).str.strip()

annotated_df['Artist'] = annotated_df['Artist'].astype(str).str.strip()
annotated_df['Title'] = annotated_df['Title'].astype(str).str.strip()

# 4. Perform the Merge
# Since both files have columns named 'Artist' and 'Title', we can merge directly on them.
final_df = pd.merge(
    my_data,
    annotated_df[['Artist', 'Title', 'Genre', 'Mood']], # Columns to add
    on=['Artist', 'Title'],                             # Match rows based on these columns
    how='left'                                          # Keep all rows from my_data
)

# 5. Check the output
print("Merged Data Preview:")
print(final_df.head())

# 6. Save the final file
final_df.to_excel("Song_Data_With_Genres.xlsx", index=False)